In [ ]:
%matplotlib notebook

import cmapPy.pandasGEXpress.parse

import cmapPy.math.fast_corr

import plotly.express as pltexpr

import sklearn.cluster

import pandas as pd

import numpy

from matplotlib import pyplot as plt

from scipy.cluster.hierarchy import dendrogram

import scipy.cluster.hierarchy

import json

In [ ]:
gctoo = cmapPy.pandasGEXpress.parse.parse("test\\2020_Q3_Achilles_CCLE_expression_r19144x1305.gctx")
print(gctoo)

In [ ]:
numpy_df = gctoo.data_df.to_numpy()

data_df_1305 = gctoo.data_df.head(3000)
transposed_data_df_1305 = data_df_1305.transpose()

result = cmapPy.math.fast_corr.fast_corr(numpy_df)

In [ ]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = numpy.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = numpy.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)
    
    print(linkage_matrix)

    # Plot the corresponding dendrogram
    r_dict = dendrogram(linkage_matrix, **kwargs)

    
    
    return linkage_matrix, list(map(int, r_dict["ivl"])), r_dict["leaves"]

# from https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html#sphx-glr-auto-examples-cluster-plot-agglomerative-dendrogram-py

In [ ]:
fig, axs = plt.subplots(3,1)

In [ ]:
#AffinityProp = sklearn.cluster.AffinityPropagation(affinity = "precomputed").fit(result)
AffinityProp = sklearn.cluster.AffinityPropagation(affinity = "euclidean", random_state = 0).fit(transposed_data_df_1305.to_numpy())


print(AffinityProp)

In [ ]:
print(list(AffinityProp.labels_))
print(len(AffinityProp.labels_))

In [ ]:
print(len(numpy.zeros(len(AffinityProp.labels_))))


In [ ]:
#result_df = pd.DataFrame(result)
result_df = data_df_1305

num_col = len(result_df.columns)
num_row = len(result_df.index)

print("num_col, num_row")
print(num_col)
print(num_row)

labels_df = pd.DataFrame(AffinityProp.labels_)
labels_df.rename(columns = {0:num_col}, inplace = True)
labels_df.index = result_df.columns

transposed_labels_df = labels_df.transpose()
transposed_labels_df.columns = result_df.columns

transposed_labels_df.insert(num_col, num_col, 20000)

print(result_df)
print(labels_df)
print(transposed_labels_df)

column_label_df = result_df.join(labels_df)

label_df = column_label_df.append(transposed_labels_df, ignore_index=True)

print(label_df)

col_sorted = label_df.sort_values(num_col)

sorted_df = col_sorted.sort_values(num_row, axis = 1)

print("sorted_df")
print(sorted_df)

sorted_df = sorted_df.drop(num_row, axis = 0)
sorted_df = sorted_df.drop(num_col, axis = 1)

print(sorted_df)



In [ ]:
print(result[24])

In [ ]:
testing_numpy = result_df.iloc[:, 24]
testing_numpy = testing_numpy.to_numpy()
testing_numpy_2 = (result_df.iloc[:, 46]).to_numpy()

empty_arr = []

empty_arr.append(testing_numpy)
empty_arr.append(testing_numpy_2)

print(empty_arr)

In [ ]:
cluster_centers = []

print(AffinityProp.cluster_centers_indices_)
print(len(AffinityProp.cluster_centers_indices_))
for i in range (0, len(AffinityProp.cluster_centers_indices_)):
    cluster_column = (result_df.iloc[:, AffinityProp.cluster_centers_indices_[i]]).to_numpy()
    cluster_centers.append(cluster_column)

In [ ]:
print(cluster_centers[0])

In [ ]:
ward_super_clusters = sklearn.cluster.AgglomerativeClustering(affinity = "euclidean", n_clusters = len(AffinityProp.cluster_centers_indices_), linkage = "average", compute_full_tree = True, compute_distances = True).fit(cluster_centers)

In [ ]:
print(len(cluster_centers))

print(len(AffinityProp.cluster_centers_indices_))

In [ ]:
pd.DataFrame(ward_super_clusters.children_)

In [ ]:
super_linkage_matrix, super_dendro_labels, super_leaves = plot_dendrogram(ward_super_clusters, distance_sort = "ascending", ax=axs[0])
plt.show()

In [ ]:
print(super_dendro_labels)

super_dendro_labels_index = []

for i in range(0, len(super_dendro_labels)):
    super_dendro_labels_index.append(super_dendro_labels.index(i))
    

print(super_dendro_labels_index)
print(AffinityProp.cluster_centers_indices_)

In [ ]:
print(label_df)

#list comperhension 
#then add col


for i in range(0, num_col + 1):
    val = int(label_df.iloc[num_row, i])
    if val != 20000 and val != numpy.NaN:
        label_df.iloc[num_row, (i)] = super_dendro_labels_index[val]
        label_df.iloc[i, num_col] = super_dendro_labels_index[val]
    
    
    
super_col_sorted = label_df.sort_values(num_col)

super_sorted_df = super_col_sorted.sort_values(num_row, axis = 1)

print(super_col_sorted)
print(super_sorted_df)

with_labels_super_sorted_df = super_sorted_df

super_sorted_df = super_sorted_df.drop(num_col, axis = 1)
super_sorted_df = super_sorted_df.drop(num_row, axis = 0)


print(super_sorted_df)

In [ ]:
#df_super_linkage_matrix[4] == numpy.NaN
#current_sub_df = df_super_linkage_matrix.loc[df_super_linkage_matrix[0] == current_sub]
#if current_sub_df.shape[0] != 1:
    #current_sub_df = current_sub_df.loc[current_sub_df[4] == numpy.NaN]
    #if current_sub_df.shape[0] != 1:
        #current_sub_df = current_sub_df.loc[current_sub_df[5] == numpy.NaN]
    
#print(current_sub_df)



In [ ]:
df_super_linkage_matrix = pd.DataFrame(super_linkage_matrix) 

In [ ]:
while df_super_linkage_matrix[2][len(df_super_linkage_matrix) - 1] > 4:
    df_super_linkage_matrix[2] = df_super_linkage_matrix[2] / 2

In [ ]:
print(df_super_linkage_matrix)

In [ ]:
 for i in range(0, len(df_super_linkage_matrix)):
        zero_leaf = df_super_linkage_matrix.iloc[i, 0]
        one_leaf = df_super_linkage_matrix.iloc[i, 1]
        if zero_leaf < len(super_dendro_labels_index):
            df_super_linkage_matrix.at[i, 0] = super_dendro_labels_index[int(zero_leaf)]
        if one_leaf < len(super_dendro_labels_index):
            df_super_linkage_matrix.at[i, 1] = super_dendro_labels_index[int(one_leaf)]

In [ ]:

df_super_linkage_matrix[4] = numpy.NaN
df_super_linkage_matrix[5] = numpy.NaN

#adding 4 to dist values of app super matrix 
df_super_linkage_matrix[2] = df_super_linkage_matrix[2] + 5

In [ ]:
print(df_super_linkage_matrix)

for row in df_super_linkage_matrix.iterrows():
    print(row)

In [ ]:
total_linkage_matrix = df_super_linkage_matrix

In [ ]:
sorted_with_labels = label_df
print(sorted_with_labels)

In [ ]:
result_df.loc[:, "ACH-001163"]

In [ ]:
abnormal_dist = []

In [ ]:
def prepare_where(df_super_linkage_matrix, current_sub):
    try:
        where = df_super_linkage_matrix.loc[df_super_linkage_matrix[0] == current_sub]
        if where.shape[0] != 1:
            where = where.loc[where[4] == numpy.NaN]
            if where.shape[0] != 1:
                where = where.loc[where[5] == numpy.NaN]
        where_col = 0
        not_where = 1
        where.index[0]
        
    except IndexError:
        where = df_super_linkage_matrix.loc[df_super_linkage_matrix[1] == current_sub]
        if where.shape[0] != 1:
            where = where.loc[where[4] == numpy.NaN]
            if where.shape[0] != 1:
                where = where.loc[where[5] == numpy.NaN]
        where_col = 1
        not_where = 0
        
    if current_sub == 59:
        print("WHERE")
        print(where)
        
    return where, where_col, not_where

In [ ]:
#probably best to put all the handy values into a dict or something so I don't need to have a large amount of arguments

def prepare_sub_matrix(len_sub, df_sub_linkage_matrix, handy_values):
    for i in range(0, handy_values['len_sub']):
        zero_leaf = df_sub_linkage_matrix.loc[i, 0]
        one_leaf = df_sub_linkage_matrix.loc[i, 1]
        if zero_leaf >= handy_values['n_sub_samples']:
            df_sub_linkage_matrix.at[i, 0] = zero_leaf + handy_values['len_super']  + handy_values['new_location']

        else:       
            df_sub_linkage_matrix.at[i, 4] = result_df.columns.get_loc(sub_cluster.index[int(zero_leaf)])       
            df_sub_linkage_matrix.at[i, 0] = handy_values['new_cluster']
            handy_values['new_cluster'] += 1

        if one_leaf >= handy_values['n_sub_samples']:
            df_sub_linkage_matrix.at[i, 1] = one_leaf  + handy_values['len_super'] + handy_values['new_location']


        else:
            df_sub_linkage_matrix.at[i, 5]  = result_df.columns.get_loc(sub_cluster.index[int(one_leaf)])
            df_sub_linkage_matrix.at[i, 1] = handy_values['new_cluster']
            handy_values['new_cluster'] += 1

    #set the greatest value = to 1
    #that is the sample name that was replaced so this is done to aviod dupes
    
    for i in range(0, len(df_sub_linkage_matrix)):
        if df_sub_linkage_matrix.loc[i, 0] == handy_values['len_super']:
            df_sub_linkage_matrix.at[i, 0] = handy_values['current_sub']
            

        if df_sub_linkage_matrix.loc[i, 1] == handy_values['len_super']:
            df_sub_linkage_matrix.at[i, 1] = handy_values['current_sub']
        
    #change index to be index of where it is going
    df_sub_linkage_matrix.index = df_sub_linkage_matrix.index + handy_values['new_location']
    
    
    return df_sub_linkage_matrix
    

In [ ]:
def prepare_super(df_super_linkage_matrix, handy_values):
        #change super cluster so sub cluster can be added 
    for i in range(0, len_super):
        
        zero_leaf = df_super_linkage_matrix.loc[i, 0]
        one_leaf = df_super_linkage_matrix.loc[i, 1]

        if zero_leaf >= n_super_samples:
            if zero_leaf - n_super_samples < new_location:
                df_super_linkage_matrix.at[i, 0] = df_super_linkage_matrix.loc[i, 0] + len_sub
            else:
                df_super_linkage_matrix.at[i, 0] = df_super_linkage_matrix.loc[i, 0] + (2 * len_sub) 

        if one_leaf >= n_super_samples:
            if one_leaf - n_super_samples < new_location:
                df_super_linkage_matrix.at[i, 1] = df_super_linkage_matrix.loc[i, 1] + len_sub
            else:
                df_super_linkage_matrix.at[i, 1] = df_super_linkage_matrix.loc[i, 1] + (2 * len_sub) 
                
    #drop where row and change indexs to prepare for append 
    df_super_linkage_matrix.drop(new_location, inplace = True)
    
    
    return df_super_linkage_matrix

In [ ]:
transposed_sorted_with_labels = sorted_with_labels.transpose()
(transposed_sorted_with_labels.loc[transposed_sorted_with_labels[num_row] == 0.0]).transpose()

In [ ]:
for current_sub in range(0, len(super_dendro_labels_index)):
    new_sub_cluster = (transposed_sorted_with_labels.loc[transposed_sorted_with_labels[num_row] == current_sub])
    sub_cluster = sorted_with_labels.loc[sorted_with_labels[num_col] == current_sub]
    print("CURRENT SUB")
    print(current_sub)
    print("old")
    print(sub_cluster)
    print("new")
    print(new_sub_cluster)

In [ ]:
for current_sub in range(0, len(super_dendro_labels_index)):
    #get all the values in the current sub cluster
    
    print("CURRENT SUB")
    print(current_sub)
    
    
    sub_cluster = (transposed_sorted_with_labels.loc[transposed_sorted_with_labels[num_row] == current_sub])
    
    df_super_linkage_matrix = total_linkage_matrix 

    #make them dataframes since they are better to work with
    #df_super_linkage_matrix = pd.DataFrame(super_linkage_matrix) 
    #df_super_linkage_matrix[4] = numpy.NaN
    #df_super_linkage_matrix[5] = numpy.NaN



    #get where the sub cluster in in the super cluster matrix
    where_col = 5
    not_where = 5

    #and df_super_linkage_matrix[5] == -1

    where, where_col, not_where = prepare_where(df_super_linkage_matrix, current_sub)

    print("WHERE")
    print(where)
    
    if sub_cluster.shape[0] != 1:
    
        

        
        # run ward
        ward_sub_cluster = sklearn.cluster.AgglomerativeClustering(affinity = "euclidean", n_clusters = len(sub_cluster)-1, linkage = "average", compute_full_tree = True, compute_distances = True).fit(sub_cluster)


        sub_linkage_matrix, sub_dendro_labels, sub_leaves = plot_dendrogram(ward_sub_cluster, distance_sort = "ascending", ax= axs[1])
        plt.show()
        
        
        #make them dataframes since they are better to work with
        df_sub_linkage_matrix = pd.DataFrame(sub_linkage_matrix) 



        #n_samples 
        n_super_samples = len(df_super_linkage_matrix) + 1
        n_sub_samples = len(df_sub_linkage_matrix) + 1

        len_super = len(df_super_linkage_matrix)
        len_sub = len(df_sub_linkage_matrix)


        new_location = where.index[0]
        new_cluster = len_super

        #changing values in sub cluster matrix so it can be merged into super cluster 



        df_sub_linkage_matrix[4] = numpy.NaN
        df_sub_linkage_matrix[5] = numpy.NaN

        while df_sub_linkage_matrix[2][len(df_sub_linkage_matrix) - 1] > 4:
            df_sub_linkage_matrix[2] = df_sub_linkage_matrix[2] / 2

        print(df_sub_linkage_matrix)
        
        handy_values = {
            'new_location':new_location,
            'new_cluster':new_cluster,
            'current_sub':current_sub,
            'len_super':len_super,
            'len_sub':len_sub,
            'n_sub_samples':n_sub_samples,
            'n_super_samples':n_super_samples
        }


        df_sub_linkage_matrix = prepare_sub_matrix(len_sub, df_sub_linkage_matrix, handy_values)
        
        #modify where line and add it to end of sub cluster matrix 
        where.at[new_location, where_col] =  new_location + len_super +  (2 * len_sub) 

        print("n_super_samples")
        print(n_super_samples)

        if where.loc[new_location, not_where] >= n_super_samples:
                if where.loc[new_location, not_where] - n_super_samples < new_location:
                    where.at[new_location, not_where] = where.loc[new_location, not_where] +len_sub
                else:
                    where.at[new_location, not_where] = where.loc[new_location, not_where] + (2 * len_sub) 

        where.at[new_location, 3] =  where.loc[new_location, 3] + n_sub_samples


        print("WHERE")
        print(where)

        #a really dumb way to rename the index
        #.setindex needs a col to set as the index 
        total_sub = pd.DataFrame(where.to_numpy(), index = [where.index[0] + n_sub_samples-1])

        total_sub = df_sub_linkage_matrix.append(total_sub)


        df_super_linkage_matrix = prepare_super(df_super_linkage_matrix, handy_values)


        print("REMOVING")
        print(new_location)

        print("TOTAL SUB")
        print(total_sub)

        print("NEW LOCATION")
        print(new_location)
        print("N_SUB_SAMPLES")
        print(n_sub_samples)
        print("len_sub")
        print(len_sub)
        print("len_super")
        print(len_super)



        print("CHANGE TO INDEX")
        print(list(range(0, new_location)) + list(range(new_location + len_sub + 1 , len_super + len_sub - 1)))
        df_super_linkage_matrix.index = list(range(0, new_location)) + list(range(new_location + len_sub + 1 , len_super + len_sub))



        #merge then sort by index 
        new_linkage_matrix = df_super_linkage_matrix.append(total_sub)
        new_linkage_matrix.sort_index(inplace = True)

        print(new_linkage_matrix)

        total_linkage_matrix = new_linkage_matrix
        
    else:
        new_linkage_matrix[where_col + 4][where.index[0]] = sub_cluster.index[0]
    
    
    
    

print(new_linkage_matrix)
scipy.cluster.hierarchy.is_valid_linkage(new_linkage_matrix)

In [ ]:
result_df.columns.get_loc('ACH-001097')

In [ ]:
for row in new_linkage_matrix.iterrows():
    print(row)

In [ ]:
#a strange way to get all the values that are not null
no_nan_4 = new_linkage_matrix.loc[new_linkage_matrix[4] > -1]
new_0 = pd.DataFrame(no_nan_4[4])
new_0.columns = [0]
print(new_0)

#a strange way to get all the values that are not null
no_nan_5 = new_linkage_matrix.loc[new_linkage_matrix[5] > -1]
new_1 = pd.DataFrame(no_nan_5[5])
new_1.columns = [1]
print(new_1)


print(new_0.sort_values(0))
print(new_1.sort_values(1))

In [ ]:
new_linkage_matrix.update(new_0)
new_linkage_matrix.update(new_1)
print(new_linkage_matrix)

In [ ]:
new_linkage_matrix = new_linkage_matrix.drop(4, axis = 1)
new_linkage_matrix = new_linkage_matrix.drop(5, axis = 1)
print(new_linkage_matrix)

In [ ]:
#one of two checks that my code often fails, see which row causes the failure 

#_check_hierarchy_uses_cluster_before_formed

Z = numpy.asarray(new_linkage_matrix, order='c')

n = Z.shape[0] + 1
for i in range(0, n - 1):
    if Z[i, 0] >= n + i or Z[i, 1] >= n + i:
        print(i)
        print(Z[i, 0])
    else:
        print(False)

In [ ]:
#one of two checks that my code often fails, see which row causes the failure 

#check_hierarchy_uses_cluster_more_than_once

n = Z.shape[0] + 1
chosen = set([])
for i in range(0, n - 1):
    if (Z[i, 0] in chosen) or (Z[i, 1] in chosen) or Z[i, 0] == Z[i, 1]:
        print(i)
        print(Z[i, 0])
        print(True)
    chosen.add(Z[i, 0])
    chosen.add(Z[i, 1])
    print(False)

print(chosen)

for i in range(0, 2607):
    if i not in chosen:
        print(i)

In [ ]:
for row in new_linkage_matrix.iterrows():
    print(row)

In [ ]:
#new_linkage_matrix.at[354, 1] = 1388.0

In [ ]:
childs = []

for i in range(0, len(new_linkage_matrix)):
    childs.append((int(new_linkage_matrix.loc[i, 0]), int(new_linkage_matrix.loc[i, 1])))



In [ ]:
len(childs)

In [ ]:
 # create the counts of samples under each node
counts = numpy.zeros(len(childs))
print(counts)
n_samples = len(new_linkage_matrix) + 1
for i, merge in enumerate(childs):
    current_count = 0
    for child_idx in merge:
        if child_idx < n_samples:
            current_count += 1  # leaf node
        else:
            current_count += counts[child_idx - n_samples]
    counts[i] = current_count
        
print(counts)

In [ ]:
counts_df = pd.DataFrame(counts)
counts_df.columns = [3]
print(counts_df)

In [ ]:
new_linkage_matrix.update(counts_df)

In [ ]:
print(new_linkage_matrix)

In [ ]:
scipy.cluster.hierarchy.is_valid_linkage(new_linkage_matrix)

In [ ]:
r_dict2 = dendrogram(new_linkage_matrix, distance_sort = "ascending", ax= axs[2])
plt.show()

In [ ]:
df_r_dict2_labels = r_dict2["leaves"]

In [ ]:
print(r_dict2["leaves"])

In [ ]:
print(len(df_r_dict2_labels))

In [ ]:
print(r_dict2["ivl"][2])

super_and_sub_dendro_labels_index = []

for i in range(0, len(r_dict2["ivl"])):
    super_and_sub_dendro_labels_index.append(df_r_dict2_labels.index(i))
    

print(super_and_sub_dendro_labels_index)

In [ ]:


df_super_and_sub_dendro_labels_index = pd.DataFrame(super_and_sub_dendro_labels_index)
df_super_and_sub_dendro_labels_index.rename(columns = {0:num_col}, inplace = True)
df_super_and_sub_dendro_labels_index.index = result_df.columns

sas_transposed_labels_df = df_super_and_sub_dendro_labels_index.transpose()
sas_transposed_labels_df.columns = result_df.columns

sas_transposed_labels_df.insert(num_col, num_col, 20000)

print(result_df)
print(df_super_and_sub_dendro_labels_index)
print(sas_transposed_labels_df)


#change so only col are changed
#sas_column_label_df = result_df.join(df_super_and_sub_dendro_labels_index)
sas_column_label_df = result_df

sas_label_df = sas_column_label_df.append(sas_transposed_labels_df)

print("sas_label_df")
print(sas_label_df)

#change so only col are changed
#sas_col_sorted = sas_label_df.sort_values(num_col)
sas_col_sorted = sas_label_df

sas_sorted_df = sas_col_sorted.sort_values(num_col, axis = 1)

print("sas_col_sorted")
print("sas_sorted_df")
print(sas_col_sorted)
print(sas_sorted_df)

with_labels_sas_sorted_df = sas_sorted_df

sas_sorted_df = sas_sorted_df.drop(num_col, axis = 0)
sas_sorted_df = sas_sorted_df.drop(num_col, axis = 1)

final_output_test = sas_sorted_df.to_numpy()

print("sas_sorted_df")
print(sas_sorted_df)

In [ ]:
fig_super = pltexpr.imshow(sas_sorted_df.to_numpy(), title="graph of sub clusters")
fig_super.show()

In [ ]:
print(with_labels_sas_sorted_df)

In [ ]:
print(with_labels_sas_sorted_df.loc[:, 'ACH-000001'])

In [ ]:
print(result_df.loc[:, 'ACH-000001'])

In [ ]:
print(super_sorted_df)

In [ ]:
fig_super = pltexpr.imshow(super_sorted_df.to_numpy(), title="graph of grouped data done by using df then reps Hierarchical Clustered")
fig_super.show()

In [ ]:
#fig_super_and_sub = pltexpr.imshow(final_sorted_df.to_numpy(), title="graph of sub and super clustered")
#fig_super_and_sub.show()

In [ ]:
from scipy.cluster import hierarchy
def getNewick(node, newick, parentdist, leaf_names):
    if node.is_leaf():
        return "%s:%.2f%s" % (leaf_names[node.id], parentdist - node.dist, newick)
    else:
        if len(newick) > 0:
            newick = "):%.2f%s" % (parentdist - node.dist, newick)
        else:
            newick = ");"
        newick = getNewick(node.get_left(), newick, node.dist, leaf_names)
        newick = getNewick(node.get_right(), ",%s" % (newick), node.dist, leaf_names)
        newick = "(%s" % (newick)
        return newick
#from https://stackoverflow.com/questions/28222179/save-dendrogram-to-newick-format/31878514#31878514 

tree = hierarchy.to_tree(new_linkage_matrix,False)
newick = getNewick(tree, "", tree.dist, r_dict2["leaves"])

In [ ]:
print(numpy.flip(list(sas_sorted_df.columns))[0])

In [ ]:
r_dict2["leaves"][0]

In [ ]:
row_metadata = gctoo.row_metadata_df
row_metadata = row_metadata.head(3000)
print(row_metadata)

In [ ]:
row_labels = list(range(0, (len(data_df_1305.index))))

In [ ]:
print(data_df_1305.index)

In [ ]:
flip_row_metadata_names_df = pd.DataFrame(numpy.flip(list(row_labels)))
flip_row_metadata_names_df.index = data_df_1305.index

print(flip_row_metadata_names_df)
print(row_metadata)

row_metadata = row_metadata.join(flip_row_metadata_names_df)

print(row_metadata)

sorted_row_metadata_df = row_metadata.sort_values(0)

sorted_row_metadata_df.drop(0, axis = 1, inplace = True)

print(sorted_row_metadata_df)

In [ ]:
col_metadata = gctoo.col_metadata_df

In [ ]:
flipped_super_and_sub_dendro_labels_index = [len(super_and_sub_dendro_labels_index) - x for x in super_and_sub_dendro_labels_index]

print(flipped_super_and_sub_dendro_labels_index)

flip_col_metadata_names_df = pd.DataFrame(flipped_super_and_sub_dendro_labels_index)
flip_col_metadata_names_df.index = result_df.columns

col_metadata[1305] = flip_col_metadata_names_df

sorted_col_metadata_df = col_metadata.sort_values(1305)

sorted_col_metadata_df.drop(1305, axis = 1, inplace = True)

print(sorted_col_metadata_df)

In [ ]:
none_sorted_col_metadata_df = sorted_col_metadata_df.where(sorted_col_metadata_df.notnull(), None)

print("none_sorted_col_metadata_df: {}".format(none_sorted_col_metadata_df))

In [ ]:
def create_col_and_row_template(isstring, field):
    col_and_row_template = {
            "squished": False,
            "inlineTooltip": False,
            "tooltip": True,
            "highlightMatchingValues": False,
            "colorBarSize": 12,
            "stackedBar": False,
            "display": [
                "text"
            ],
            "selectionColor": "rgb(182,213,253)",
            "colorByField": None,
            "fontField": None,
            "barColor": "#bdbdbd",
            "barSize": 40,
            "autoscaleAlways": False,
            "minMaxReversed": False,
            "field": field,
            "size": {
                "height": 10
            }
        }
    if isstring:
        col_and_row_template["maxTextWidth"] = 0
        
    return col_and_row_template

In [ ]:
def create_col_and_row_metadata_template(isstring, field, array):
    col_and_row_metadata_template = {
        "properties": {"morpheus.dataType": "number"},
            "name": field,
            "array": array
          }
    if isstring:
        col_and_row_metadata_template["properties"]["morpheus.dataType"] = "string"
        
    return col_and_row_metadata_template

In [ ]:
final_sorted_df_numpy = final_output_test
flipped = numpy.flip(final_sorted_df_numpy)

#final_sorted_df_numpy = final_sorted_df.to_numpy()
#flipped = numpy.flip(final_sorted_df_numpy)

template_file = open("template.json", "r")
json_object = json.load(template_file)
template_file.close()

json_object["columnDendrogram"] = newick
json_object["rowDendrogram"] = newick

del json_object["rowDendrogram"]

dataset = json_object["dataset"]

dataset["rows"] = len(flipped)
dataset["columns"] = len(flipped[0])

print("flipped.tolist()")
print(flipped.tolist()[0])


dataset["seriesArrays"] = [flipped.tolist()]
r_dict2["leaves"].reverse()

#reset columns and rows only necessary for now will change template so it is not later
json_object["columns"] = []
json_object["rows"] = []

json_object["columns"].append(create_col_and_row_template(False,"tree_id"))
json_object["columns"].append(create_col_and_row_template(True, "cid"))

json_object["rows"].append(create_col_and_row_template(False,"tree_id"))
json_object["rows"].append(create_col_and_row_template(True, "rid"))


dataset["rowMetadataModel"]["vectors"] = []
dataset["columnMetadataModel"]["vectors"] = []

dataset["rowMetadataModel"]["vectors"].append(create_col_and_row_metadata_template(False, "tree_id", r_dict2["leaves"])) 
dataset["columnMetadataModel"]["vectors"].append(create_col_and_row_metadata_template(False, "tree_id", r_dict2["leaves"])) 

sas_flipped_index = numpy.flip(list(sas_sorted_df.index))
sas_flipped_col = numpy.flip(list(sas_sorted_df.columns))
                                                                                                       
dataset["rowMetadataModel"]["vectors"].append(create_col_and_row_metadata_template(True, "rid", sas_flipped_index.tolist())) 
dataset["columnMetadataModel"]["vectors"].append(create_col_and_row_metadata_template(True, "cid", sas_flipped_col.tolist()))

for col in none_sorted_col_metadata_df.iteritems():
    json_object["columns"].append(create_col_and_row_template(True, col[0]))
    dataset["columnMetadataModel"]["vectors"].append(create_col_and_row_metadata_template(True, col[0], col[1].to_list())) 

for row in gctoo.row_metadata_df.iteritems(): 
    json_object["rows"].append(create_col_and_row_template(True, row[0]))
    dataset["rowMetadataModel"]["vectors"].append(create_col_and_row_metadata_template(True, row[0], row[1].to_list())) 
    
    
output_file = open("super_and_sub_file.json", "w")
json.dump(json_object, output_file)
output_file.close()

In [ ]:
for i in range(0, len(result_df.columns)):
    result_df_val = (result_df.loc[result_df.index[i], result_df.columns[i]])
    sas_sorted_df_val = (sas_sorted_df.loc[result_df.index[i], result_df.columns[i]])
    if result_df_val != sas_sorted_df_val:
        print(result_df_val)
        print(sas_sorted_df_val)
        print(i)
        

In [ ]:
X = numpy.array([[1, 2, 3, 3], [1, 4, 5, 4], [1, 0, 5, 0],[4, 2, 3, 2], [4, 4, 5, 4], [4, 0, 3, 0]])

In [ ]:
df_x = pd.DataFrame([[1, 2, 3, 3], [1, 4, 5, 4], [1, 0, 5, 0],[4, 2, 3, 2], [4, 4, 5, 4], [4, 0, 3, 0]])

df_x = df_x.transpose()

print(df_x)

In [ ]:
clustering = sklearn.cluster.AffinityPropagation(affinity = "euclidean", random_state = 0).fit(df_x.to_numpy())
#clustering = sklearn.cluster.AgglomerativeClustering(affinity = "euclidean", n_clusters = 2, linkage = "average", compute_full_tree = True, compute_distances = True).fit(df_x.to_numpy())

In [ ]:
clustering.cluster_centers_indices_

In [ ]:
col_metadata_df.equals(none_sorted_col_metadata_df)

In [ ]:
gctoo.row_metadata_df